In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
from tqdm import tqdm

# Config
BASE_MODEL = "Salesforce/codegen-350M-mono"
LORA_PATH = "./lora-adapter"
TEST_PATH = "./1-encoder-data/test/test.jsonl"
OUTPUT_PATH = "./predictions.txt"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Load base model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32)
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)
tokenizer.padding_side = "left"


# Inject LoRA adapter
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.to(DEVICE)
model.eval()

# Load test data
test_data = []
with open(TEST_PATH, "r", encoding="utf-8") as f:
    for line in f:
        test_data.append(json.loads(line))

from torch.utils.data import DataLoader

BATCH_SIZE = 16  
MAX_INPUT_LEN = 512
MAX_NEW_TOKENS = 256

#Tokenize without tensors for correct padding later
inputs = [tokenizer(example["input"], truncation=True, max_length=MAX_INPUT_LEN) for example in test_data]

# Pad using tokenizer logic (respects padding_side)
batch_encoding = tokenizer.pad(inputs, padding=True, return_tensors="pt")

input_ids = batch_encoding["input_ids"]
attention_mask = batch_encoding["attention_mask"]

# Create batches
dataloader = DataLoader(list(zip(input_ids, attention_mask)), batch_size=BATCH_SIZE)

# Predict
with open(OUTPUT_PATH, "w", encoding="utf-8") as out_file:
    for batch in tqdm(dataloader, desc="Generating predictions"):
        input_ids_batch, attention_mask_batch = [x.to(DEVICE) for x in batch]

        with torch.no_grad():
            output_tokens = model.generate(
                input_ids=input_ids_batch,
                attention_mask=attention_mask_batch,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=0.0,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

        for input_ids_i, generated_i in zip(input_ids_batch, output_tokens):
            decoded = tokenizer.decode(generated_i, skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids_i, skip_special_tokens=True)
            prediction = decoded[len(input_text):].strip() if decoded.startswith(input_text) else decoded.strip()
            out_file.write(prediction + "\n")


print(f" Finished predictions:  {OUTPUT_PATH}")


c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Using device: cuda


c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Chris\miniconda3\envs\deepseek13b\lib\site-packages\transformers\modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly al

 Finished predictions:  ./predictions.txt
